In [1]:
# test full training of maskcnn_polished_with_rcnn_k_bl
from os.path import join

import numpy as np

import torch

from thesis_v2 import dir_dict
from thesis_v2.data.prepared import combine_two_separate_datasets

from thesis_v2.data.prepared.cadena_plos_cb19 import get_data

from thesis_v2.training_extra.maskcnn_like.opt import get_maskcnn_v1_opt_config
from thesis_v2.training_extra.maskcnn_like.training import (train_one,
                                                            partial)

from thesis_v2.models.maskcnn_polished_with_rcnn_k_bl.builder import (
    gen_maskcnn_polished_with_rcnn_k_bl,
    load_modules
)

In [2]:
def pad_nan(y_this):
    pad = np.full_like(y_this, fill_value=np.nan)
    return np.concatenate([pad, y_this, pad], axis=1)


def train_one_maskcnn_polished_with_rcnn_k_bl(
    split_seed,
    model_seed,
    act_fn,
    loss_type,
    input_size,
    out_channel,
    num_layer,
    kernel_size_l1,
    pooling_ksize,
    scale, scale_name,
    smoothness, smoothness_name,
    pooling_type,
    n_timesteps,
):
    
    load_modules()
    datasets = get_data(
         px_kept=80, final_size=input_size, scale=0.5,
        seed=split_seed
    )

    datasets = {
        'X_train': datasets[0].astype(np.float32),
        'y_train': pad_nan(datasets[1]),
        'X_val': datasets[2].astype(np.float32),
        'y_val': pad_nan(datasets[3]),
        'X_test': datasets[4].astype(np.float32),
        'y_test': pad_nan(datasets[5]),
    }
    
    datasets_combined = dict()
    for zzz in ['train', 'val', 'test']:
        datasets_combined[f'X_{zzz}'], datasets_combined[f'y_{zzz}'] = combine_two_separate_datasets(
            x1=datasets[f'X_{zzz}'], y1=datasets[f'y_{zzz}'],
            x2=datasets[f'X_{zzz}'], y2=datasets[f'y_{zzz}'],
        )
        
    del datasets

    def gen_cnn_partial(input_size_cnn, n):
        return gen_maskcnn_polished_with_rcnn_k_bl(
                                    input_size=input_size_cnn,
                                    num_neuron=n,
                                    out_channel=out_channel,
                                    kernel_size_l1=kernel_size_l1,  # (try 5,9,13)
                                    kernel_size_l23=3,
                                    act_fn=act_fn,
                                    pooling_ksize=pooling_ksize,  # (try, 1,3,5,7)
                                    pooling_type=pooling_type,  # try (avg, max)  # looks that max works well here?
                                    num_layer=num_layer,
                                    n_timesteps=n_timesteps,
                                    factored_constraint=None,
                                    blstack_pool_ksize=1,
                                    blstack_pool_type=None,
                                    acc_mode='cummean',
                                    bn_after_fc=False,
                                    ff_1st_block=True,
                                    )

    opt_config_partial = partial(get_maskcnn_v1_opt_config,
                                 scale=scale,
                                 smoothness=smoothness,
                                 group=0.0,
                                 loss_type=loss_type,
                                 )
    
    result = train_one(
        arch_json_partial=gen_cnn_partial,
        opt_config_partial=opt_config_partial,
        datasets=datasets_combined,
        key=f'debug/test_full_training_maskcnn_polished_with_rcnn_k_bl/ff_1st_block_cb19_data-handle_nan-nan_y_2/{model_seed}',
        show_every=100,
        max_epoch=40000,
        model_seed=model_seed,
        return_model=False,
        extra_params={
            # reduce on batch axis
            'eval_fn': {
                'yhat_reduce_axis': 1,
                'handle_nan': True,
            }
        },
        val_test_every=100,
        print_model=True,
        handle_nan=True,
    )
    
    return result['stats_best']['stats']['test']['corr_mean'], result['stats_best']['stats']['test']['corr']

In [3]:
maskcnn_param_template = {
    'out_channel': 16,
    'num_layer': 3,
    'kernel_size_l1': 9,
    'pooling_ksize': 3,
    'pooling_type': 'avg',
    'model_seed': 0,
    'split_seed': 0,
}

maskcnn_param_regular = {
    **maskcnn_param_template,
    **{
        'act_fn': 'relu',
        'loss_type': 'mse',
        'smoothness': 0.000005,
        # 0.000005 gives 0.505823562521005
        # 0.00005 gives 0.5051972572416786
        # 0.0000005 gives 0.5043545818049401
        
        'smoothness_name': '0.000005',
        
        # with smoothness set to 0.00005,
        # scale=0.01 gives 0.505823562521005
        # scale=0.1 gives 0.48046038315427236
        # scale=0.001 gives 0.4979968643549016
        
        # my previous hyperparameters worked the best!!!
        
        'scale': 0.01,
        'scale_name': '0.01',
        'input_size': 40,
        'n_timesteps': 1,
    }
}

print(train_one_maskcnn_polished_with_rcnn_k_bl(**maskcnn_param_regular))

# 27629 = 1*2 + 16*1*9*9 + 16*2 + 2*(16*16*3*3+16*16*3*3+4*16*2) + 79*(14*14 + 16 + 1) 

{'final_act', 'pooling', 'bn_output', 'fc'}
['conv0', 'bl_stack.layer_list.0.b_conv', 'bl_stack.layer_list.1.b_conv']
neurons with NaN mean on train 460/690
num_param 101222


/my_data/thesis-yimeng-v2/thesis_v2/training_extra/maskcnn_like/training.py:122: RuntimeWarning: Mean of empty slice
  resp_mean = np.nanmean(resp_train, axis=0)


JSONNet(
  (moduledict): ModuleDict(
    (accumulator): RecurrentAccumulator()
    (act0): ReLU()
    (bl_stack): BLConvLayerStack(
      (layer_list): ModuleList(
        (0): BLConvLayer(
          (b_conv): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        )
        (1): BLConvLayer(
          (b_conv): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        )
      )
      (bn_layer_list): ModuleList(
        (0): BatchNorm2d(16, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
        (1): BatchNorm2d(16, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
      )
      (capture_list): ModuleList(
        (0): Identity()
        (1): Identity()
      )
      (input_capture): Identity()
      (act_fn): ReLU(inplace=True)
      (pool): Identity()
    )
    (bn0): BatchNorm2d(16, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
    (bn_input): BatchNorm2d(1, eps=0.001, momentum

/opt/conda/envs/leelab/lib/python3.7/site-packages/numpy/core/_methods.py:217: RuntimeWarning: Degrees of freedom <= 0 for slice
  keepdims=keepdims)
/opt/conda/envs/leelab/lib/python3.7/site-packages/numpy/core/_methods.py:186: RuntimeWarning: invalid value encountered in true_divide
  arrmean, rcount, out=arrmean, casting='unsafe', subok=False)
/opt/conda/envs/leelab/lib/python3.7/site-packages/numpy/core/_methods.py:209: RuntimeWarning: invalid value encountered in true_divide
  ret = ret.dtype.type(ret / rcount)


val metric {'loss': 0.3389215111732483, 'loss_no_reg': 0.32218363881111145, 'corr': [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, -0.03636707680099892, 0.0, -0.020413369225385246, 0.06073123693879514, -0.021479562041815572, 0.0, -0.09870826539279147, -0.07834875537889552, 0.0, 0.0, 0.0177130386117605, 0.041849598062992564, -0.06731290651712972, -0.11887825966036311, -0.009518470645299398, 0.0, 0.028041060202139572, 0.0, 0.001592085063129433, 0.0, 0.0, 0.0, 0.00

...

========starting epoch 900==========
900-0, train loss 0.23788219690322876
train loss 0.23788219690322876
val metric {'loss': 0.2786326467990875, 'loss_no_reg': 0.2559348940849304, 'corr': [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.2120893250312825, 0.4789638343550163, 0.45732163165766765, 0.4741063934601635, 0.6190770633998511, 0.49015578051472614, 0.4095354375479642, 0.42224528945690026, 0.35095949836652596, 0.15098224177544442, 0.4967176187839152, 0.22117453

========starting epoch 1000==========
1000-0, train loss 0.2533186972141266
train loss 0.2533186972141266
val metric {'loss': 0.2785372108221054, 'loss_no_reg': 0.25583475828170776, 'corr': [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.21043736688355336, 0.47964662054418905, 0.4553593733357974, 0.47400845823922233, 0.6181165682006405, 0.48838932817487724, 0.4081191892030307, 0.4231338744744878, 0.35256341030218, 0.1505621307604341, 0.49487941671587415, 0.221142012

========starting epoch 1100==========
1100-0, train loss 0.25004300475120544
train loss 0.25004300475120544
val metric {'loss': 0.27865890711545943, 'loss_no_reg': 0.255940705537796, 'corr': [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.21215925422516282, 0.4790827603858054, 0.4563319681423692, 0.4736957742247585, 0.6173545419321871, 0.4880913510857923, 0.40829097183347335, 0.4228491232463637, 0.3515661937167254, 0.15064048385804676, 0.49549392028425, 0.2201448437

========starting epoch 1200==========
1200-0, train loss 0.23523740470409393
train loss 0.23523740470409393
val metric {'loss': 0.2788208082318306, 'loss_no_reg': 0.2561190724372864, 'corr': [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.21126941911089664, 0.48055806356712155, 0.4563146791703597, 0.47323657671772407, 0.6192274295071645, 0.4887320403895937, 0.4104420526439122, 0.4222400338939006, 0.3534665248732069, 0.14963725115588675, 0.4951352847044386, 0.2207439

(0.16883447360612713, [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.23592947237937478, 0.4561970631518701, 0.49720548715054425, 0.46579347282407935, 0.6285641133060949, 0.4787283675632525, 0.3762041171736556, 0.4531953858226444, 0.365285048756529, 0.19702320666575576, 0.5249123319113853, 0.24541565682380878, 0.5468790502203786, 0.2607058321908122, 0.5510359332858378, 0.2538998316039865, 0.32128052404063845, 0.21582967724361452, 0.31341037944579536, 0.4937975650301

In [4]:
# get the non-zero part.

this_one = [0.23592947237937478, 0.4561970631518701, 0.49720548715054425, 0.46579347282407935, 0.6285641133060949, 0.4787283675632525, 0.3762041171736556, 0.4531953858226444, 0.365285048756529, 0.19702320666575576, 0.5249123319113853, 0.24541565682380878, 0.5468790502203786, 0.2607058321908122, 0.5510359332858378, 0.2538998316039865, 0.32128052404063845, 0.21582967724361452, 0.31341037944579536, 0.4937975650301336, 0.3688098587598391, 0.5753078597137913, 0.33343169715036913, 0.4120528537169197, 0.4753433871328141, 0.3165326966028097, 0.49659239087433826, 0.5093093609589944, 0.6304704504454012, 0.44840786938185195, 0.4067865515234319, 0.44677121339006104, 0.6262018351595476, 0.48805819459854266, 0.4821929939687711, 0.5149162837784577, 0.3639228726191686, 0.5635083066536976, 0.574407621750155, 0.10778243933288667, 0.3723581354944518, 0.5165685297366056, 0.3673401768388914, 0.38031987081323543, 0.27518511791360967, 0.4162631090980857, 0.6580962296001239, 0.5167972732366033, 0.4769674122325876, 0.5436021625925727, 0.6842659399932325, 0.5342313803030401, 0.5786599129320735, 0.5184245468066728, 0.6685862891424701, 0.6693327104524599, 0.7045444176335829, 0.4444833020738341, 0.7555575285756652, 0.31254825668937186, 0.4839315198954421, 0.40997701347800986, 0.45934663252627184, 0.5510269197050813, 0.5194179413461211, 0.536603304718139, 0.5680714928026163, 0.386102593575497, 0.5260578326550988, 0.6229461268277187, 0.5302984954360923, 0.7514428215149566, 0.5144409867531888, 0.40642986763705485, 0.5633006903297052, 0.4782485233298662, 0.6401131098752788, 0.5952337391279214, 0.17536917084444237, 0.5097334986223361, 0.5506129850287476, 0.5859429638917861, 0.6230253881209383, 0.7286742523722458, 0.6588576868434621, 0.7692132825895569, 0.6858623249557563, 0.6073463861721239, 0.4398440229166493, 0.4886950604366719, 0.7301730048908083, 0.6498716277788315, 0.42365237356066343, 0.5880431255350725, 0.5745422979478191, 0.6409226903374625, 0.5046820146949318, 0.7836377299790649, 0.49853142574945203, 0.531519199167061, 0.5490909263070289, 0.45969061335831884, 0.5706121750147052, 0.6964302269832467, 0.5786746460869869, 0.6798180342507223, 0.601341631025554, 0.5915548980983878, 0.5647380789012381, 0.4241957218863619, 0.556233478352308, 0.4023814951634964, 0.6581567233926009, 0.6657672846062025, 0.6028911717525909]

this_one_another = [
    
    0.2362744658392616, 0.4567260458970454, 0.496800979729788, 0.46614730750316774, 0.6293577489770352, 0.47931562307980125, 0.3756772117707322, 0.45218748772952655, 0.3112760170932417, 0.19771938654097854, 0.5246294649157104, 0.2452733209332343, 0.5498467597145303, 0.2625310056947124, 0.5514531518678564, 0.2534946556766334, 0.323357715384433, 0.2139550512844388, 0.3150469076342759, 0.49324407225690936, 0.36816237504968324, 0.5754545258819723, 0.33221556930679236, 0.4110445051760744, 0.4756102721072088, 0.31488157454306165, 0.4960944693641084, 0.5108171065965657, 0.6306066397240317, 0.44765917246450243, 0.40760518036716675, 0.4456117158784845, 0.6259783007878779, 0.4880285086191872, 0.48208580110442345, 0.5139478423263202, 0.32226479351662835, 0.56414470133833, 0.5734145406927156, 0.09881994058906506, 0.3711259447097694, 0.5168618424367709, 0.36790310660562126, 0.38017960850438187, 0.27555098337415945, 0.4154316977736741, 0.6582544627536118, 0.5174301929835861, 0.4759070058589836, 0.5433571980797312, 0.6856194431782927, 0.533604197796157, 0.5793630517085298, 0.5179172961341751, 0.6686011543226799, 0.6688801085941197, 0.7046252272945763, 0.4454140375565379, 0.7551617295500227, 0.31416876853571263, 0.48284546703287895, 0.41162670429582165, 0.45959024442975915, 0.5508980808660386, 0.5199765993637971, 0.5366740409256172, 0.5683347762588669, 0.3846546864375078, 0.5249239144886063, 0.6228403716020675, 0.5304613482650306, 0.7511923390292589, 0.5129522890379214, 0.40736312117363527, 0.5624758342193694, 0.47874020101096276, 0.6396532408747877, 0.5953011641288132, 0.1582292879430591, 0.5119651708885973, 0.5507615991570748, 0.5862016671344963, 0.6236298361269388, 0.7284644119789705, 0.658589379472785, 0.7683906715591182, 0.6861118474233646, 0.6073293777892885, 0.4383336450616445, 0.488420853911595, 0.7293132846673318, 0.6496246402513918, 0.42305741785941553, 0.5877272417984626, 0.5746824866589241, 0.6413676598674974, 0.5053760398836519, 0.7835498844766946, 0.49875861257003584, 0.5315388279575204, 0.5500696036104697, 0.4589068148766393, 0.5707339822648259, 0.6962802712294746, 0.5789670334462143, 0.6797181346238874, 0.6011545170115133, 0.591257106383686, 0.563659661786805, 0.42416656601160607, 0.5557455839625799, 0.4027601591835639, 0.6583239668462956, 0.6656955806789756, 0.6027497803130707
]

In [5]:
# from rcnn_basic_kriegeskorte/test_full_training_ff_1st_cb19_data-handle_nan.ipynb

that_one = [0.22290867814140142, 0.4708074534173289, 0.4748005428071667, 0.4726406797165408, 0.6261681532403353, 0.47778410642306074, 0.355276401771913, 0.4059499988799383, 0.27306687854409467, 0.2060898428331426, 0.5066163009016792, 0.21927353933146942, 0.5601374651206583, 0.2686960626493656, 0.5464271834695443, 0.2654781651778954, 0.32077724062824814, 0.23326450810839897, 0.3647180023372379, 0.47421547330506253, 0.3785651481083519, 0.5456766079217238, 0.3364432552323184, 0.4076604394937908, 0.4603158349474313, 0.3521687998817147, 0.4759313387428043, 0.5159867024433585, 0.6517636553453058, 0.44227082061214457, 0.42572542630461446, 0.46016411728227063, 0.6354862858793571, 0.48589746898023056, 0.4898000842666996, 0.5060820441937934, 0.3344468595601764, 0.5575963223930174, 0.565608854549304, 0.14956376027860313, 0.3688827327773246, 0.5157210848584867, 0.37949105748215006, 0.38131425254857293, 0.29023178616260425, 0.4092873045632762, 0.6546056913526376, 0.5105307686644647, 0.4648774654180571, 0.5389574293108751, 0.6848201217249001, 0.5474474526035918, 0.5821566244878427, 0.51226782001166, 0.6609746645934759, 0.6702189332496112, 0.713059286114458, 0.45907239875451583, 0.7417166861688765, 0.30437573690915565, 0.4779997589486579, 0.427084702651328, 0.4953790066043401, 0.519572799621041, 0.5208000639149223, 0.5429269548689853, 0.5504635125870688, 0.3740523838454645, 0.5148547506060256, 0.617953049407335, 0.5446498963984046, 0.7483751444632495, 0.5351439216693811, 0.411975940704237, 0.5626833094802879, 0.48104878226225906, 0.6523571578566921, 0.6083369240852424, 0.15578991723678842, 0.5054916925086076, 0.5579364138230724, 0.5694297383452513, 0.6215502504670586, 0.7279874841137555, 0.6224736989941038, 0.761737156238404, 0.6861978108122229, 0.6067185997711673, 0.44793477697308465, 0.5357085593856187, 0.7331999385987633, 0.6475202233616871, 0.44000115178619764, 0.5841800693523288, 0.581523834811977, 0.6218072448705786, 0.5003692096558175, 0.7780975902298893, 0.49409784451955313, 0.527406207860786, 0.5361577988956837, 0.4588214757156862, 0.5553959701146822, 0.7058145972361495, 0.5886926663525995, 0.6853363853392981, 0.5815182730815066, 0.5858139427845256, 0.5553463898266813, 0.42168832978027504, 0.5526428668634553, 0.40211562266310247, 0.6608694233816872, 0.6747063506035611, 0.59971232260701]

In [6]:
from scipy.stats import pearsonr

In [7]:
# basically the same
print(pearsonr(this_one, that_one))
print(pearsonr(this_one_another, that_one))

(0.9914922985802004, 6.547414417594228e-102)
(0.9935035660059993, 1.6661792393052394e-108)


In [8]:
np.mean(this_one), np.mean(this_one_another), np.mean(that_one)

(0.5070395720470513, 0.5059672695897114, 0.505823562521005)

In [9]:
np.std(this_one), np.std(this_one_another), np.std(that_one)

(0.1357207377288545, 0.13730773401742952, 0.13470891707205376)